# Classical Solver for QUBO Optimization

This project uses a **classical integer programming solver** to find the optimal RNA secondary structure by minimizing the QUBO (Quadratic Unconstrained Binary Optimization) Hamiltonian. The QUBO is constructed from the potential stems, pseudoknot penalties, and overlap penalties, and encodes the energy landscape of possible RNA foldings.

We use the **Google OR-Tools CP-SAT solver** to solve this problem. The solver treats each potential stem as a Boolean variable (0 = not selected, 1 = selected) and finds the combination of stems that minimizes the total energy, while automatically respecting the penalties for overlaps and pseudoknots.

**Key steps:**
1. **Preprocessing:** Extract all potential stems, pseudoknots, and overlaps from the RNA sequence.
2. **Model Construction:** Build the QUBO Hamiltonian (linear and quadratic terms) using the preprocessing results.
3. **Optimization:** Use the CP-SAT solver to find the optimal set of stems (the predicted structure).
4. **Evaluation:** Compare the predicted structure to the actual (experimentally determined) structure.

In [1]:
from preprocess_sequence import (
    actual_stems, potential_stems, potential_pseudoknots, potential_overlaps, model, energy
)
from classical_optimizer import solve_qubo_with_cpsat

# Define file paths
subdirectory = "./data"
ct_file = "bpRNA_RFAM_23352.ct.txt"
fasta_file = "bpRNA_RFAM_23352.fasta.txt"

# params
pseudoknot_penalty = 2.0


# 1. Get actual stems (ground truth)
actual = actual_stems(ct_file, fasta_file, subdirectory)
print("Actual stems (ground truth):", actual)

# 2. Get potential stems and related info
pot_stems, mu, rna, seq_len = potential_stems(fasta_file, subdirectory)

# 3. Get potential pseudoknots and overlaps
pot_pks = potential_pseudoknots(pot_stems, pseudoknot_penalty)
pot_ovs = potential_overlaps(pot_stems)

# 4. Build the QUBO model (linear and quadratic terms)
L, Q = model(pot_stems, pot_pks, pot_ovs, mu)

# 5. Solve the QUBO using the classical solver
solution, energy_pred = solve_qubo_with_cpsat(L, Q)

# 6. Extract the predicted stems from the solution
print("Actual stems:", actual)
predicted_stems = [pot_stems[i] for i, v in solution.items() if v == 1]
print("Predicted stems:", predicted_stems)


# 7. Calculate and print the energy of the actual structure
energy_actual = energy(actual, pseudoknot_penalty)
print("Energy of actual structure:", energy_actual)

# 8. Calculate and print the energy of the predicted structure
energy_predicted = energy(predicted_stems, pseudoknot_penalty)
print("Energy of predicted structure:", energy_predicted)

Actual stems (ground truth): [[2, 19, 8], [11, 33, 11], [16, 27, 3]]
[DEBUG] Number of variables: 74
[DEBUG] Number of quadratic terms: 2701
[DEBUG] Sample linear terms: [(0, 102589), (1, 13289), (2, -104310), (3, -16110), (4, 42690)]
[DEBUG] Sample quadratic terms: [((0, 1), 999944000), ((0, 2), 999912000), ((0, 3), 999872000), ((0, 4), 999904000), ((0, 5), 999872000)]
[DEBUG] Starting solver...
[DEBUG] Solver finished with a solution.
Best solution: {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 1}
Best energy: -245.021
Actual stems:

In [2]:
# Import quantum optimizer
from quantum_optimizer import build_quadratic_program, solve_with_vqe, analyze_result
import numpy as np

# Scale down the problem for quantum solver
def scale_qubo(L, Q, scale_factor=1e-6):
    """Scale down the QUBO coefficients to avoid numerical issues."""
    L_scaled = {k: v * scale_factor for k, v in L.items()}
    Q_scaled = {k: v * scale_factor for k, v in Q.items()}
    return L_scaled, Q_scaled

# Select a subset of variables for quantum solver (due to qubit limitations)
def select_subset(L, Q, n_vars=8):
    """Select a subset of variables for quantum solver."""
    # Get the most important variables based on linear terms
    important_vars = sorted(L.items(), key=lambda x: abs(x[1]), reverse=True)[:n_vars]
    selected_vars = {v[0] for v in important_vars}
    
    # Filter L and Q to only include selected variables
    L_subset = {k: v for k, v in L.items() if k in selected_vars}
    Q_subset = {k: v for k, v in Q.items() if k[0] in selected_vars and k[1] in selected_vars}
    
    return L_subset, Q_subset

# Convert integer indices to string names for Qiskit
def convert_to_string_names(L, Q):
    """Convert integer indices to string names for Qiskit."""
    L_str = {f"x_{k}": v for k, v in L.items()}
    Q_str = {(f"x_{k[0]}", f"x_{k[1]}"): v for k, v in Q.items()}
    return L_str, Q_str

# Scale and subset the problem
L_scaled, Q_scaled = scale_qubo(L, Q)
L_subset, Q_subset = select_subset(L_scaled, Q_scaled, n_vars=8)

print("Original problem size:", len(L), "variables")
print("Subset problem size:", len(L_subset), "variables")

# Convert to string names for Qiskit
L_str, Q_str = convert_to_string_names(L_subset, Q_subset)

# Solve with quantum solver
qp = build_quadratic_program(L_str, Q_str)
print("\nQUBO as QuadraticProgram:")
print(qp.export_as_lp_string())

print("\n--- Quantum VQE Solution ---")
result = solve_with_vqe(qp, max_evals=300)
analyze_result(result)

# Convert result back to original indices
quantum_solution = {int(k.split('_')[1]): v for k, v in zip(L_str.keys(), result.x)}

# Compare with classical solution for the same subset
subset_solution = {k: solution[k] for k in L_subset.keys()}
subset_energy = sum(L_subset[k] * v for k, v in subset_solution.items()) + \
                sum(Q_subset[k] * subset_solution[k[0]] * subset_solution[k[1]] 
                    for k in Q_subset.keys())

print("\n--- Comparison ---")
print("Classical solution for subset:", subset_solution)
print("Classical energy for subset:", subset_energy)
print("Quantum solution:", quantum_solution)
print("Quantum energy:", result.fval)

# Print the actual stems corresponding to the solutions
print("\n--- Stem Comparison ---")
print("Classical stems:", [pot_stems[i] for i, v in subset_solution.items() if v == 1])
print("Quantum stems:", [pot_stems[i] for i, v in quantum_solution.items() if v == 1])

Original problem size: 74 variables
Subset problem size: 8 variables

QUBO as QuadraticProgram:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 0.000192510000 x_10 - 0.000104310000 x_17 - 0.000104310000 x_2
      - 0.000163110000 x_21 - 0.000163110000 x_32 - 0.000192510000 x_43
      - 0.000133710000 x_50 - 0.000133710000 x_9 + [ 1.999384000000 x_10*x_17
      + 1.998768000000 x_10*x_2 + 1.999272000000 x_10*x_21
      + 1.999272000000 x_10*x_32 + 1.999216000000 x_10*x_43
      + 1.999328000000 x_10*x_50 + 1.999328000000 x_10*x_9
      + 1.999032000000 x_17*x_2 + 1.999428000000 x_17*x_21
      + 1.998856000000 x_17*x_32 + 1.998768000000 x_17*x_43
      + 1.999472000000 x_17*x_50 + 1.999472000000 x_17*x_9
      + 1.998856000000 x_2*x_21 + 1.998856000000 x_2*x_32
      + 1.998768000000 x_2*x_43 + 1.998944000000 x_2*x_50
      + 1.998944000000 x_2*x_9 + 1.999324000000 x_21*x_32
      + 1.999272000000 x_21*x_43 + 1.999376000000 x_21*x_5

KeyError: '2'